In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl

from celerite import GP
import eztao
# from eztao.carma import DRW_term,DHO_term, CARMA_term
from eztao.carma import *
from eztao.ts import gpSimRand,gpSimFull
from eztao.ts import drw_fit
from eztao.ts import gpSimFull, gpSimByTime
from eztao.ts.carma_fit import sample_carma
from eztao.carma import gp_psd
from eztao.ts import drw_fit
from eztao.ts import dho_fit

In [40]:
sample_pd = pd.read_csv('SRNN_dataset/drw_3881.csv')


Underdamping

In [20]:
beta_0, beta_1 = 0.0022, 0.00025

alpha1_l = []
alpha2_l = []
QPO_l = []
tau_l = []
sf_l = []
z_l = []

# under_pd = pd.DataFrame(columns = ['id', 'band','alpha1', 'alpha2', 'beta0', 'beta1', 'SF', 'redshift', 'mu'])
n = 0

id_l = []
band_l = []
alpha1_l = []
alpha2_l = []
sf_l = []
z_l = []
mu_l = []

while n<len(sample_pd):
    valid_count = 0
    for i in ['u','g','r','i','z','y']:
        alpha_1 = 2/sample_pd['tau_'+i][n]
        var = sample_pd['SF_'+i][n]**2/2
        alpha_2 = beta_0**2/(2*alpha_1*var-beta_1**2)
        if alpha_1**2-4*alpha_2<0 and alpha_2>0:
            valid_count +=1
            id_l.append(sample_pd['SDR5ID'][n])
            band_l.append(i)
            alpha1_l.append(alpha_1)
            alpha2_l.append(alpha_2)
            sf_l.append(sample_pd['SF_'+i][n])
            z_l.append(sample_pd['redshift'][n])
            mu_l.append(sample_pd['mu_'+i][n]) 
    if valid_count<6:
        del id_l[-valid_count:], band_l[-valid_count:], alpha1_l[-valid_count:], 
        alpha2_l[-valid_count:], sf_l[-valid_count:], z_l[-valid_count:], mu_l[-valid_count:]
        
    n +=1

beta0_l = [beta_0]*len(id_l)
beta1_l = [beta_1]*len(id_l)
df_under = pd.DataFrame(list(zip(id_l, band_l, alpha1_l, alpha2_l, beta0_l, beta1_l, sf_l, z_l, mu_l)), 
                       columns = ['id', 'band','alpha1', 'alpha2', 'beta0', 'beta1', 'SF', 'redshift', 'mu'])


In [23]:
df_under.to_csv('underDHO_1075.csv')

Overdamped

In [43]:
sample_pd = sample_pd.sample(400)
sample_pd = sample_pd.reset_index()

In [44]:
from sympy import *
beta_0, beta_1 = 0.003672,0.0257

id_l = []
band_l = []
alpha1_l = []
alpha2_l = []
sf_l = []
z_l = []
mu_l = []

n = 0
while n<len(sample_pd):
    print(n)
    valid_count = 0
    for i in ['u','g','r','i','z','y']:
        alpha_1= Symbol('alpha_1')
        alpha_2= Symbol('alpha_2')
        var = sample_pd['SF_'+i][n]**2/2
        tau = sample_pd['tau_'+i][n]
        solved_value = solve([tau*alpha_1 - tau**2*alpha_2 -1, 
                           (beta_0**2+beta_1**2*alpha_2)/(2*alpha_1*alpha_2)-var],
                         [alpha_1, alpha_2])
        alpha_1 = float(solved_value[1][0])
        alpha_2 = float(solved_value[1][1])
       
        if  alpha_1**2-4*alpha_2>=0:
            valid_count +=1
            id_l.append(sample_pd['SDR5ID'][n])
            band_l.append(i)
            alpha1_l.append(alpha_1)
            alpha2_l.append(alpha_2)
            sf_l.append(sample_pd['SF_'+i][n])
            z_l.append(sample_pd['redshift'][n])
            mu_l.append(sample_pd['mu_'+i][n]) 
    if valid_count<6:
        del id_l[-valid_count:], band_l[-valid_count:], alpha1_l[-valid_count:], 
        alpha2_l[-valid_count:], sf_l[-valid_count:], z_l[-valid_count:], mu_l[-valid_count:]
         
    n +=1

# print(id_l)
beta0_l = [beta_0]*len(id_l)
beta1_l = [beta_1]*len(id_l)
df_over = pd.DataFrame(list(zip(id_l, band_l, alpha1_l, alpha2_l, beta0_l, beta1_l, sf_l, z_l, mu_l)), columns = ['id', 'band','alpha1', 'alpha2', 'beta0', 'beta1', 'SF', 'redshift', 'mu'])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [46]:
df_over

,id,band,alpha1,alpha2,beta0,beta1,SF,redshift,mu
0,75622.0,u,0.301475,0.000404,0.003672,0.0257,0.335969,1.2199,20.8077
1,75622.0,g,0.281303,0.000608,0.003672,0.0257,0.284909,1.2199,20.4564
2,75622.0,r,0.469058,0.000565,0.003672,0.0257,0.228754,1.2199,20.0555
3,75622.0,i,0.533973,0.000545,0.003672,0.0257,0.218127,1.2199,19.9623
4,75622.0,z,0.427060,0.000602,0.003672,0.0257,0.232443,1.2199,19.9076
...,...,...,...,...,...,...,...,...,...
2395,72316.0,g,0.389487,0.001022,0.003672,0.0257,0.188577,0.5888,19.0579
2396,72316.0,r,0.402610,0.001117,0.003672,0.0257,0.177831,0.5888,19.0824
2397,72316.0,i,0.467047,0.001805,0.003672,0.0257,0.131955,0.5888,18.8615
2398,72316.0,z,0.510288,0.002159,0.003672,0.0257,0.116335,0.5888,18.8772


In [47]:
df_over.to_csv('SRNN_dataset/overDHO_400.csv')